In [0]:
# Comment these lines if not using Google Colaboratory
!pip install -U ray[rllib]
!pip install gputil

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gym
from gym import spaces
import numpy as np

import ray
import ray.rllib.agents.ppo as ppo
import ray.rllib.agents.dqn as dqn
import ray.rllib.agents.dqn.apex as apex
from ray.tune.logger import pretty_print

ray.shutdown()
ray.init(webui_host="127.0.0.1",ignore_reinit_error=True, log_to_driver=False)

In [0]:
# Use apex?
ape = 0
if ape:
    config = apex.APEX_DEFAULT_CONFIG.copy()
    config["min_iter_time_s"] = 2
    config["lr"] = 0.000005
    config["buffer_size"] = 200000
    config["timesteps_per_iteration"] = 8000
    config["n_step"] = 4
    niter = 30
else:
    config = dqn.DEFAULT_CONFIG.copy()
    config["schedule_max_timesteps"] = 60000
    config["timesteps_per_iteration"]: 4000
    niter = 18

# Config
#config["num_envs_per_worker"] = 8
config["num_gpus"] = 1
config["num_workers"] = 2
config["gamma"] = 0.95

if ape:
    agent = apex.ApexTrainer(config=config, env="Blackjack-v0")
else:
    agent = dqn.DQNTrainer(config=config, env="Blackjack-v0")

for i in range(niter):
    # Perform one iteration of training the policy
    result = agent.train()
    print("Iteration", i, "-- Mean Reward: ",result["episode_reward_mean"])



In [0]:
# Evaluate
env = gym.make("Blackjack-v0")

avgR = 0
N=5000
for i in range(N):
    obs = env.reset()
    done = False
    R = 0
    while not done:
        action = agent.compute_action(obs)
        obs, r, done, _ = env.step(action)
        R += r

    avgR = avgR + R

print('Evaluation mean reward:',avgR/N)